In [ ]:
!cd  F:\workspace\fascrapper\fashion_scrapper
!pip install -e .

In [1]:
from pathlib import Path
from bs4 import BeautifulSoup
from scrapper.util.web.static import find_first_parent_href
from scrapper.util.list import flatten, distinct, distinct_list_of_dicts
from scrapper.util.web.dynamic import driver as ddriver
from scrapper.brand.asos.webelements.views.Asos_Categories_Elements import Asos_Categories_Elements
from scrapper.brand.asos.webelements.views.Asos_Category_Elements import Asos_Category_Elements
from scrapper.brand.asos.webelements.views.Asos_item_Elements import Asos_Article_Elements
from default_logger.defaultLogger import defaultLogger

from random import shuffle
from tqdm.auto import tqdm
from scrapper.util.web.dynamic import driver as d_driver

from time import sleep
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from scrapper.util.web.dynamic import wait, scroll_end_of_page
from scrapper.brand.asos.Asos import Asos
from scrapper.util.list import includes_excludes_filter, flatten
from multiprocessing import Pool, freeze_support

CATEGORIES = [
    {"name": "schuhe", "includes": ["shoe"], "excludes": []},
    {"name": "hose", "includes": ["short", "jeans", "leggings", "trousers"], "excludes": []},
    {"name": "shirt", "includes": ["shirt", "skirt", "blazer", "top"], "excludes": []},
    {"name": "pullover", "includes": ["pullover", "cardigans"], "excludes": []},
    {"name": "jacke", "includes": ["coat"], "excludes": []},
    {"name": "kleid", "includes": ["dresses"], "excludes": ["kleidung"]},
    {"name": "anzug", "includes": ["suit", "overalls"], "excludes": []}
]
excludes = ["sale",  "view+all", "new-in-clothing", "accessories", "face-body", "topshop",  "back-in-stock", \
            "fashion-online-4", "curve-plus-size", "maternity", "petite", "tall", "fashion-online-12", "generic-promos", \
           "designer", "a-to-z-of-brands", "exclusives", "activewear", "co-ords", "multipacks", "bags", \
            "bras", "fashion-online-", "yoga-studio", "ski-snowboard", "running", "outdoors", "ss-fashion-trend-", \
            "gym-training", "gifts", "wedding-attire", "underwear", \
            "plus-size", "outlet-edits", "back-to-school", "modestwear", "socks-tights", "swim", "lingerie", \
            "loungewear", "-essentials", "responsible-edit", "wedding", "workwear", "jewellery", "sunglasses", "party-wear", \
           "licence"]

logger = defaultLogger("asos")
THREADS = 2


def filter_categories():
    logger.debug("Filter Categories")
    with d_driver(headless=False) as driver:
        asos = Asos(driver=driver, logger=logger)
        categories = asos.list_categories()

        filter_category = lambda category: (category["name"], [x for x in categories if
                                                               includes_excludes_filter(x["url"],
                                                                                        includes=category["includes"],
                                                                                        excludes=excludes)])
        filterd_categories = [filter_category(x) for x in CATEGORIES]
        return filterd_categories


def flatten_category(category):
    cat_name, cat_data = category
    cat_urls = [x["url"] for x in cat_data]
    return list(zip([cat_name] * len(cat_urls), cat_urls))


def flatten_categories(categories):
    return flatten([flatten_category(x) for x in categories])

def load_category(cat_data):
    category_name, category_url = cat_data
    print(category_name)
    with d_driver(headless=False) as driver:
        asos = Asos(driver=driver, logger=logger)
        logger.debug("Loading" + category_url)
        print("Loading" + category_url)
        items = asos.list_category(category_url, PAGINATE=False)
        return [{"name": category_name, "url": category_url, "items": [x]} for x in items]

def prepare_categories():
    """
    Alle Items laden (kein DL)
    :return:
    """
    category_jobs = flatten_categories(filter_categories())
    category_jobs = category_jobs[:THREADS]
    categories_data = []
    with Pool(THREADS) as p:
        r = p.map(load_category, tqdm(category_jobs, desc=f"i) List Cat. {THREADS} Threads", total=len(category_jobs)))
        categories_data.append(r)
        return flatten(categories_data)




#categories_data = flatten(prepare_categories())

In [2]:
import os
os.cpu_count()

16

In [3]:
categories_data

NameError: name 'categories_data' is not defined

In [11]:
tmp = [ {"name": "xyz", "url": "asdasd", "items": [1, 2, 3]}, 
 {"name": "xyzabc", "url": "asdasd", "items": [1, 2, 3]}, 
 {"name": "xyz", "url": "asdasd", "items": [4, 5, 6]}]

In [12]:
tmp

[{'name': 'xyz', 'url': 'asdasd', 'items': [1, 2, 3]},
 {'name': 'xyzabc', 'url': 'asdasd', 'items': [1, 2, 3]},
 {'name': 'xyz', 'url': 'asdasd', 'items': [4, 5, 6]}]

In [16]:
from collections import defaultdict
results = defaultdict(lambda: [])

In [21]:
tmp

[{'name': 'xyz', 'url': 'asdasd', 'items': [1, 2, 3]},
 {'name': 'xyzabc', 'url': 'asdasd', 'items': [1, 2, 3]},
 {'name': 'xyz', 'url': 'asdasd', 'items': [4, 5, 6]}]

In [17]:
for name in tmp:
    results[x["name"]].append(x)
    

In [20]:
results.items()

dict_items([('xyz', [{'name': 'xyz', 'url': 'asdasd', 'items': [1, 2, 3]}, {'name': 'xyz', 'url': 'asdasd', 'items': [4, 5, 6]}]), ('xyzabc', [{'name': 'xyzabc', 'url': 'asdasd', 'items': [1, 2, 3]}])])